# Defining Release Scenarios

The whole point of `GasDispersion.jl` is to model chemical releases into the atmosphere, to do this one needs to have a defined chemical release scenario. 

The default is to assume the release is a jet of material issuing from a single point, emitting a constant mass emission rate for a given release duration. The coordinate system is such that the release point is at *x=0*, *y=0*, *z=h* where *x* is the downwind distance, *y* the cross wind distance, and *z* the elevation. 

<a title="Mbeychok, CC BY-SA 3.0 &lt;http://creativecommons.org/licenses/by-sa/3.0/&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Gaussian_Plume.png"><img alt="Gaussian Plume" src="https://upload.wikimedia.org/wikipedia/commons/1/10/Gaussian_Plume.png"></a>

<a href="https://commons.wikimedia.org/wiki/File:Gaussian_Plume.png">Mbeychok</a>, <a href="http://creativecommons.org/licenses/by-sa/3.0/">CC BY-SA 3.0</a>, via Wikimedia Commons

The gas dispersion models require the following parameters to be defined for a given scenario:

- mass emission rate, kg/s
- release duration, s
- jet diameter, m
- jet velocity, m/s
- jet density, kg/m³
- release pressure, Pa
- release temperature, K
- release height, m
- windspeed, m/s
- ambient density, kg/m³
- ambient pressure, Pa
- ambient temperature, K
- pasquill stability class

Standard methods given:
+ *Lee's Loss Prevention in the Process Industries*
+ CCPS *Guidelines for Consequence Analysis of Chemical Releases*
+ CCPS *Guidelines for Chemical Process Quantitative Risk Analysis*
+ CCPS *Guidelines for Vapour Cloud Explosion, Pressure Vessel Burst, BLEVE and Flash Fire Hazards*

and many other texts on chemical process safety can be used to determine all of these parameters and a `Scenario` created directly.

In [2]:
using GasDispersion

In [3]:
scenario = Scenario(
    1.0,   # mass emission rate, kg/s
    10.0,  # release duration, s
    0.25,  # jet diameter, m
    15.67, # jet velocity, m/s
    1.3,   # jet density, kg/m³
    101325,# release_pressure, Pa
    450,   # release temperature, K
    1.0,   # release height, m
    1.5,   # windspeed, m/s
    1.225, # ambient density, kg/m³
    101325,# ambient pressure, Pa
    298.15,# ambient temperature, K
    "F",   # pasquill stability class
)

Release scenario:
    mass_emission_rate: 1.0 kg/s 
    release_duration: 10.0 s 
    jet_diameter: 0.25 m 
    jet_velocity: 15.67 m/s 
    jet_density: 1.3 kg/m^3 
    release_pressure: 101325 Pa 
    release_temperature: 450 K 
    release_height: 1.0 m 
    windspeed: 1.5 m/s 
    ambient_density: 1.225 kg/m^3 
    ambient_pressure: 101325 Pa 
    ambient_temperature: 298.15 K 
    pasquill_gifford: F  
    

Or by passing a dictionary with the known values (missing values will be set to `missing`)

In [4]:
d = Dict([
    :release_pressure => 101325,   # Pa
    :release_temperature => 298.15,# K
    :ambient_density => 1.225,     # kg/m^3
    :ambient_pressure => 101325,   # Pa
    :ambient_temperature => 298.15,# K
])

scenario = Scenario(d)

Release scenario:
    mass_emission_rate: missing kg/s 
    release_duration: missing s 
    jet_diameter: missing m 
    jet_velocity: missing m/s 
    jet_density: missing kg/m^3 
    release_pressure: 101325.0 Pa 
    release_temperature: 298.15 K 
    release_height: missing m 
    windspeed: missing m/s 
    ambient_density: 1.225 kg/m^3 
    ambient_pressure: 101325.0 Pa 
    ambient_temperature: 298.15 K 
    pasquill_gifford: missing  
    

Or by passing an existing scenario, and replacing some parameters using keyword arguments

In [5]:
scenario = Scenario(scenario; mass_emission_rate=1.0, release_duration=10.0)

Release scenario:
    mass_emission_rate: 1.0 kg/s 
    release_duration: 10.0 s 
    jet_diameter: missing m 
    jet_velocity: missing m/s 
    jet_density: missing kg/m^3 
    release_pressure: 101325.0 Pa 
    release_temperature: 298.15 K 
    release_height: missing m 
    windspeed: missing m/s 
    ambient_density: 1.225 kg/m^3 
    ambient_pressure: 101325.0 Pa 
    ambient_temperature: 298.15 K 
    pasquill_gifford: missing  
    

`GasDispersion` also comes with some basic tools for constructing simple release scenarios for common situations

## Flow through an orifice
The easiest type of release scenario to model is flow through an orifice or hole, for example a puncture in the side of a process vessel.

### Liquid Jet
A liquid jet through a small hole can be modeled by the Bernoulli equation, with the jet velocity given by

$$ u_j = c_d  \sqrt{ 2 \left( P_r - P_a \over \rho_l \right) } $$

where
- $u_j$ is the jet velocity, m/s
- $c_d$ the discharge coefficient, unitless
- $P_r$ the release pressure, Pa, i.e. the pressure inside the vessel at the hole
- $P_a$ the ambient pressure, Pa, i.e. the pressure outside the hole
- $\rho_l$ the liquid density, kg/m³

the mass emission rate is then given by
$$ \dot{m} = \rho_l A_h u_j $$

where $A_h$ is the cross-sectional area of the hole, in m²

#### Example

This scenario is adapted from CCPS *Guidelines for Consequence Analysis of Chemical Releases*, CCPS, pg 40.

Suppose a leak of a liquid from a storage tank through a hole. The tank headspace is at 0.1 barg and there is 2 m of liquid head above the hole. The liquid has a density of 490 kg/m³, and is in thermal equilibrium with the environment. Assume a circular hole with a discharge coefficient of 0.63 and a diameter of 1cm.

For ambient conditions we assume the atmosphere is at standard conditions of 1atm and 25°C, with a windspeed of 1.5m/s and class F stability.

Assumptions:
- discharge coefficient, $c_d = 0.63 $
- diameter of the hole, $d_j = 0.010 \textrm{m}$
- liquid density, $\rho_j = \rho_l = 490 \textrm{kg/m}^3$
- liquid temperature, $T_r = 298.15 \textrm{K}$
- standard pressure, $P_a = 101325 \textrm{Pa}$
- standard temperature, $T_a = 298.15 \textrm{K}$
- density of air at standard conditions, $\rho_a = 1.225 \textrm{kg/m}^3$
- ambient windspeed, $u = 1.5 \textrm{m/s}$

In [6]:
# What we know
cd = 0.63
dⱼ = 0.010    #m
ρⱼ = 490.0    #kg/m³
Tᵣ = 298.15   #K
Pₐ = 101325.0 #Pa
Tₐ = 298.15   #K
ρₐ = 1.225    #kg/m³
u  = 1.5      #m/s
stability = "F"

"F"

What we need to solve for is
- $P_r$, the release pressure, Pa,
- $u_j$, the jet velocity, m/s
- $\dot{m}$, the mass emission rate, kg/s

The pressure immediately inside the liquid discharge is the sum of the headspace pressure and the pressure due to the 2 m of liquid head and is

$$ P_r = P_{headspace} + \rho_j g h $$
$$ P_r = (10000 Pa + 101325 Pa) + 490 kg/m³ \cdot 9.80616 m/s² \cdot 2 m $$

In [7]:
g  = 9.80616  # gravitational acceleration, m/s²

Pᵣ = (10000 + Pₐ) + ρⱼ*g*2

120935.0368

The jet velocity comes from the Bernoulli equation

$$ u_j = c_d  \sqrt{ 2 \left( P_r - P_a \over \rho_l \right) } $$

In [8]:
uⱼ = cd * √( 2*( (Pᵣ - Pₐ) / ρⱼ) )

5.636333880812954

We are assuming a circular hole so the cross sectional area is just

$$ A_h = {\pi \over 4} d_j^2 $$

In [9]:
Aₕ = (π/4)*dⱼ^2

7.853981633974483e-5

and the mass emission rate is given by the equation
$$ \dot{m} = \rho_l A_h u_j $$

In [10]:
m = ρⱼ*Aₕ*uⱼ

0.21691154763598

we can now put this all together to make a release scenario, ready for modeling:

In [11]:
s = Dict([
    :mass_emission_rate => m,
    :jet_diameter => dⱼ,
    :jet_velocity => uⱼ,
    :jet_density => ρⱼ,
    :release_pressure => Pᵣ,
    :release_temperature => Tᵣ,
    :release_height => 0.0,
    :windspeed => u,
    :ambient_density => ρₐ,
    :ambient_pressure => Pₐ,
    :ambient_temperature => Tₐ,
    :pasquill_gifford => stability
])

scenario_1 = Scenario(s)

Release scenario:
    mass_emission_rate: 0.21691154763598 kg/s 
    release_duration: missing s 
    jet_diameter: 0.01 m 
    jet_velocity: 5.636333880812954 m/s 
    jet_density: 490.0 kg/m^3 
    release_pressure: 120935.0368 Pa 
    release_temperature: 298.15 K 
    release_height: 0.0 m 
    windspeed: 1.5 m/s 
    ambient_density: 1.225 kg/m^3 
    ambient_pressure: 101325.0 Pa 
    ambient_temperature: 298.15 K 
    pasquill_gifford: F  
    

The helper function `scenario_builder` does basically this, which simplifies the workflow immensly. All one needs to do is supply some basic information and, using default values for ambient conditions, a release scenario is generated.

In [12]:
scenario_2 = scenario_builder(Pᵣ, Tᵣ; windspeed=u, stability=stability, 
             model=:jet, phase=:liquid, liquid_density=ρⱼ, 
             hole_diameter=dⱼ, discharge_coeff=0.63)

Release scenario:
    mass_emission_rate: 0.21691154763598 kg/s 
    release_duration: missing s 
    jet_diameter: 0.01 m 
    jet_velocity: 5.636333880812954 m/s 
    jet_density: 490.0 kg/m^3 
    release_pressure: 120935.0368 Pa 
    release_temperature: 298.15 K 
    release_height: 0.0 m 
    windspeed: 1.5 m/s 
    ambient_density: 1.225 kg/m^3 
    ambient_pressure: 101325.0 Pa 
    ambient_temperature: 298.15 K 
    pasquill_gifford: F  
    

In [13]:
scenario_1 == scenario_2

true

### Ideal Gas Jet

TODO

### Two-Phase Jet

TODO